In [1]:
# -*- coding: utf-8 -*-

參考資料：
 1. from
  -http://strworkplace.hopto.org:8794/notebooks/%E6%95%B8%E6%93%9A%E7%A7%91%E5%AD%B8%E6%9C%9F%E6%9C%AB%E5%A0%B1%E5%91%8A/%E6%B0%A3%E8%B1%A1%E8%B3%87%E6%96%99/%E7%88%AC%E8%9F%B2_%E8%A7%80%E6%B8%AC%E8%B3%87%E6%96%99_strong_%E5%88%9D%E5%A7%8B-Copy1_%E4%BE%9D%E7%85%A7%E5%B9%B4%E6%9C%88%E6%97%A5%E3%80%81%E5%9C%B0%E9%BB%9E%E6%8A%93%E8%B3%87%E6%96%99_%E5%8A%A0%E5%85%A50%E8%AE%80%E5%8F%96%E6%AA%94%E6%A1%88.ipynb
    
 + JSON viewer
  - http://jsonviewer.stack.hu/
 + pyhton print flush。想弄單行輸出
  -  http://www.revotu.com/python3-print-function-keyword-arguments.html
  - http://www.revotu.com/how-to-flush-output-of-python-print.html
  - https://stackoverflow.com/questions/5598181/python-multiple-prints-on-the-same-line

思考: 
  + 資料列表重複讀取的問題
   - 還要同一個測站，有點難
  + 直接塞 日期清單進 dict 吧

In [2]:
import pandas
import json

In [23]:
import os.path

###### 參數

In [10]:
# 定義月份最大天
def YMtoMonDays(year, mon, d = 0): # ****-**
    year = int(year)
    mon = int(mon) + d
    if mon < 1:
        mon = mon + 12
    elif mon > 12:
        mon = mon - 12
    monthMaxDayList = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    if mon != 2:
        days = monthMaxDayList[mon-1]
    else:
        if year%4 != 0:
            days = 28
        elif (year%4 == 0) and (year%100 != 0):
            days = 29
        elif (year%100 == 0) and (year%400 != 0):
            days = 28
        elif (year%400 == 0):
            days = 29
    return days

###### 輸入

In [11]:
def GetCWBData(locationCity, station, stname, yearMon, dayList): #no, name, timeYM,timeD 
    #return
    doOrNot = True
    # 重複資料重複使用
    if yearMon in temp_df_dict.keys():
        if station in temp_df_dict[yearMon].keys():
            monthTable = temp_df_dict[yearMon][station]
            doOrNot = False
    else:
        temp_df_dict[yearMon] = dict()
    if doOrNot:
        # 資料表網址
        search_url = base_url+"MonthDataController.do?command=viewMain&station="+station+"&stname="+stname+"&datepicker="+yearMon
        # 抓資料表
        df = pandas.read_html(search_url)
        monthTable = df[1][1:]
        #清除標頭亂碼
        header = []
        for te in monthTable.iloc[0]:
            temp = te.split(')')[-1]
            #print(temp)
            header.append(temp)
        #設定 header
        #monthTable.columns = monthTable.iloc[0]
        monthTable.columns = header
        # 重設標頭
        monthTable = monthTable.reindex(monthTable.index.drop(1))
        
        #儲存
        temp_df_dict[yearMon][station] = monthTable
    ##### OLD
    #存進dict
    # 確認dict["年月"]存在
    if yearMon not in data_dict.keys():
        data_dict[yearMon] = dict()
    # 確認dict["地點"]存在 
    if yearMon not in data_dict[yearMon].keys():
        data_dict[yearMon][locationCity] = dict()
    # 依照指定日期存資料
    for d in dayList:
        #確認dict["日"]存在
        if d not in data_dict[yearMon][locationCity].keys():
            data_dict[yearMon][locationCity][d] = dict()
        #儲存資料
        for dayType in needDataList[1:]:
            #print(d, type(d))
            data_dict[yearMon][locationCity][d][dayType] = monthTable[dayType][d+1]    
    return 

In [12]:
def RunData(earthquakeDate, locationCity, stationDict):
    #例外處理
    if "桃園" in locationCity:
        temp = locationCity.split("縣")
        locationCity = temp[0] + "市" 
    # 日期清單
    dateSeq = list()
    # 取得時間
    temp = earthquakeDate.split("-")
    year = "%0.2d"%(int(temp[0]))
    month = "%0.2d"%(int(temp[1]))
    day = "%0.2d"%(int(temp[2]))
    # 取得觀測站名稱及編號
    station = stationDict[locationCity]["stationNo"]
    stname = stationDict[locationCity]["stationName"]
    #日期設置 - 當月份
    startDay = int(day) - dayInterval
    endDay = int(day) + dayInterval
    #開始有跨月問題: 
    #若不跨月
    if (int(day) - dayInterval > 0 ) and (int(day) + dayInterval <=  YMtoMonDays(year, month)):
        # dict 對應名稱，對網頁也是
        yearMon = "%s-%0.2d"%(year, int(month))
        dayList = list(range(startDay, endDay+1))
        # 擷取資料
        GetCWBData(locationCity, station, stname, yearMon, dayList)
        for d in dayList:
            dateSeq.append(yearMon + '-' + str(d))
    # 跨月不跨年
    else:
        #print(year,"-",month)
        yearMonList = list()
        dayListList = list()
        if (int(day) - dayInterval <= 0 ): #print("往過去跨月")
            #原本月份 - 未來
            yearMon = "%s-%0.2d"%(year, int(month))
            dayList = list(range(1, endDay+1))
            
            yearMonList.append(yearMon)
            dayListList.append(dayList)
            #新月份 - 過去
                #跨年與否
            if month == "01":
                month_n = 12
                year_n = "%0.2d"%(int(year)-1)
            else:
                month_n = int(month) - 1
                year_n = year
            yearMon_n = "%s-%0.2d"%(year_n, month_n)
            monthMaxDay_n = YMtoMonDays(year, month, -1)
            dayList_n = list(range(monthMaxDay_n + startDay, monthMaxDay_n+1))
            
            yearMonList.append(yearMon_n)
            dayListList.append(dayList_n)
        elif (int(day) + dayInterval > YMtoMonDays(year, month)): #print("往未來跨月")
            #原本月份 - 過去
            yearMon = "%s-%0.2d"%(year, int(month))
            dayList = list(range(startDay, YMtoMonDays(year, month)+1))
            
            yearMonList.append(yearMon)
            dayListList.append(dayList)
            #新月份 - 未來
                #跨年與否
            if month == "12":
                month_n = 1
                year_n = "%0.2d"%(int(year)+1)
            else:
                month_n = int(month) + 1
                year_n = year
            yearMon_n = "%s-%0.2d"%(year_n, month_n)
            dayList_n = list(range(1, endDay - YMtoMonDays(year, month) + 1))
            
            yearMonList.append(yearMon_n)
            dayListList.append(dayList_n)
        
        #print(yearMonList)
        #print(dayListList)    
        #執行撈取
        for i in range(len(yearMonList)):
            GetCWBData(locationCity, station, stname, yearMonList[i], dayListList[i])
    
        for i in range(len(yearMonList)):
            yearMon_t = yearMonList[i]
            dayList_t = dayListList[i]
            for d in dayList_t:
                dateSeq.append(yearMon_t + '-' + str(d))
    return dateSeq

# TEST

### 與現有資料更新

###### 清空暫存空間

BUG : 
 - T ?

# MAIN

In [ ]:
#變數
base_url = "https://e-service.cwb.gov.tw/HistoryDataQuery/"
specificWeatherDataJSON = "SpecificWeatherData.json" #天氣資料儲存位置
earthquakeDataJSON = "earthquakeData.json"
stationDictJson = "cityStationData.json"
inputDataJSON = "SpecificWeatherData.json"
# - 欲抓取天氣資料
needDataList = ["ObsTime", 'Temperature', 'RH', "Precp", "StnPres"]
# - 時間區間寬度
dayInterval = 3
# 
data_dict = dict()
# 單一次 data frame 儲存使用
temp_df_dict = dict()

In [ ]:
#檔案讀取
with open(stationDictJson, 'r') as inputfile:
    stationDict =  json.load(inputfile)
with open(earthquakeDataJSON, 'r') as inputfile:
    earthquakeDataDict =  json.load(inputfile)

In [17]:
for year in sorted(earthquakeDataDict.keys()):
    print(year)
    for num in sorted(earthquakeDataDict[year].keys()):
        temp = earthquakeDataDict[year][num]
        #print(num, temp, flush = False)
        print("\r\r\r\r\r"+num, end="", flush = False)
        locationCity = temp['location']
        earthquakeDate = "%s-%s"%(year, temp['date'])
        #print(earthquakeDate, locationCity)
        #正式
        dateSeq = RunData(earthquakeDate, locationCity, stationDict)
        earthquakeDataDict[year][num]['dateSeq'] = dateSeq

2011
{'date': '01-02', 'location': '宜蘭縣', 'strength': '3.9'}
{'date': '01-10', 'location': '宜蘭縣', 'strength': '3.7'}
{'date': '01-11', 'location': '宜蘭縣', 'strength': '4.4'}
{'date': '01-15', 'location': '宜蘭縣', 'strength': '3.3'}
{'date': '01-25', 'location': '臺中市', 'strength': '4.0'}
{'date': '01-25', 'location': '嘉義縣', 'strength': '3.7'}
{'date': '01-28', 'location': '高雄市', 'strength': '4.2'}
{'date': '02-01', 'location': '花蓮縣', 'strength': '4.9'}
{'date': '02-01', 'location': '花蓮縣', 'strength': '5.5'}
{'date': '02-03', 'location': '花蓮縣', 'strength': '3.7'}
{'date': '02-05', 'location': '宜蘭縣', 'strength': '4.5'}
{'date': '02-07', 'location': '花蓮縣', 'strength': '3.7'}
{'date': '02-07', 'location': '花蓮縣', 'strength': '3.9'}
{'date': '02-07', 'location': '花蓮縣', 'strength': '4.1'}
{'date': '02-07', 'location': '花蓮縣', 'strength': '4.6'}
{'date': '02-08', 'location': '臺中市', 'strength': '3.6'}
{'date': '02-09', 'location': '南投縣', 'strength': '3.2'}
{'date': '02-26', 'location': '雲林縣', 'stren

{'date': '10-31', 'location': '宜蘭縣', 'strength': '5.0'}
{'date': '11-04', 'location': '花蓮縣', 'strength': '4.0'}
{'date': '11-05', 'location': '花蓮縣', 'strength': '3.8'}
{'date': '11-06', 'location': '嘉義縣', 'strength': '4.0'}
{'date': '11-09', 'location': '宜蘭縣', 'strength': '3.8'}
{'date': '11-16', 'location': '花蓮縣', 'strength': '3.8'}
{'date': '11-16', 'location': '花蓮縣', 'strength': '3.4'}
{'date': '11-17', 'location': '宜蘭縣', 'strength': '4.2'}
{'date': '11-19', 'location': '花蓮縣', 'strength': '4.3'}
{'date': '11-19', 'location': '嘉義市', 'strength': '3.1'}
{'date': '11-19', 'location': '臺東縣', 'strength': '3.9'}
{'date': '11-21', 'location': '花蓮縣', 'strength': '4.3'}
{'date': '11-22', 'location': '臺南市', 'strength': '3.8'}
{'date': '12-02', 'location': '臺東縣', 'strength': '4.8'}
{'date': '12-02', 'location': '宜蘭縣', 'strength': '4.2'}
{'date': '12-03', 'location': '花蓮縣', 'strength': '3.9'}
{'date': '12-04', 'location': '宜蘭縣', 'strength': '4.9'}
{'date': '12-05', 'location': '臺東縣', 'strength':

{'date': '07-07', 'location': '臺東縣', 'strength': '3.6'}
{'date': '07-10', 'location': '宜蘭縣', 'strength': '4.6'}
{'date': '07-17', 'location': '花蓮縣', 'strength': '3.6'}
{'date': '07-18', 'location': '花蓮縣', 'strength': '3.2'}
{'date': '07-20', 'location': '新北市', 'strength': '2.7'}
{'date': '07-21', 'location': '花蓮縣', 'strength': '3.5'}
{'date': '07-23', 'location': '宜蘭縣', 'strength': '4.3'}
{'date': '07-29', 'location': '臺東縣', 'strength': '3.1'}
{'date': '07-31', 'location': '桃園縣', 'strength': '3.3'}
{'date': '08-04', 'location': '屏東縣', 'strength': '4.3'}
{'date': '08-14', 'location': '花蓮縣', 'strength': '5.2'}
{'date': '08-15', 'location': '花蓮縣', 'strength': '4.1'}
{'date': '08-15', 'location': '臺東縣', 'strength': '3.1'}
{'date': '08-15', 'location': '花蓮縣', 'strength': '4.1'}
{'date': '08-16', 'location': '花蓮縣', 'strength': '3.6'}
{'date': '08-17', 'location': '花蓮縣', 'strength': '4.6'}
{'date': '08-17', 'location': '花蓮縣', 'strength': '3.8'}
{'date': '08-18', 'location': '花蓮縣', 'strength':

{'date': '06-29', 'location': '宜蘭縣', 'strength': '3.5'}
{'date': '06-29', 'location': '花蓮縣', 'strength': '5.7'}
{'date': '07-03', 'location': '宜蘭縣', 'strength': '3.9'}
{'date': '07-06', 'location': '臺東縣', 'strength': '4.1'}
{'date': '07-11', 'location': '花蓮縣', 'strength': '3.4'}
{'date': '07-15', 'location': '臺東縣', 'strength': '4.6'}
{'date': '07-15', 'location': '花蓮縣', 'strength': '5.7'}
{'date': '07-16', 'location': '花蓮縣', 'strength': '5.5'}
{'date': '07-21', 'location': '花蓮縣', 'strength': '4.3'}
{'date': '07-23', 'location': '臺中市', 'strength': '3.2'}
{'date': '07-24', 'location': '花蓮縣', 'strength': '4.8'}
{'date': '07-24', 'location': '花蓮縣', 'strength': '5.0'}
{'date': '07-24', 'location': '花蓮縣', 'strength': '4.3'}
{'date': '07-25', 'location': '臺東縣', 'strength': '3.9'}
{'date': '08-09', 'location': '花蓮縣', 'strength': '3.8'}
{'date': '08-13', 'location': '花蓮縣', 'strength': '3.9'}
{'date': '08-24', 'location': '南投縣', 'strength': '4.7'}
{'date': '08-29', 'location': '花蓮縣', 'strength':

{'date': '05-16', 'location': '臺南市', 'strength': '4.0'}
{'date': '05-17', 'location': '花蓮縣', 'strength': '3.4'}
{'date': '05-18', 'location': '宜蘭縣', 'strength': '3.0'}
{'date': '05-21', 'location': '花蓮縣', 'strength': '5.9'}
{'date': '05-21', 'location': '花蓮縣', 'strength': '3.5'}
{'date': '05-24', 'location': '花蓮縣', 'strength': '4.2'}
{'date': '05-24', 'location': '宜蘭縣', 'strength': '4.1'}
{'date': '05-24', 'location': '宜蘭縣', 'strength': '3.1'}
{'date': '05-25', 'location': '臺東縣', 'strength': '5.0'}
{'date': '05-28', 'location': '臺東縣', 'strength': '3.8'}
{'date': '05-29', 'location': '臺東縣', 'strength': '4.7'}
{'date': '05-31', 'location': '臺東縣', 'strength': '4.4'}
{'date': '05-31', 'location': '新竹縣', 'strength': '4.2'}
{'date': '05-31', 'location': '宜蘭縣', 'strength': '3.8'}
{'date': '06-06', 'location': '花蓮縣', 'strength': '4.0'}
{'date': '06-15', 'location': '花蓮縣', 'strength': '4.6'}
{'date': '06-15', 'location': '花蓮縣', 'strength': '3.6'}
{'date': '06-18', 'location': '宜蘭縣', 'strength':

ConnectionResetError: [WinError 10054] 遠端主機已強制關閉一個現存的連線。

In [24]:
def updateAndSaveJSON(nameJSON, dictJSON):
    #更新與儲存
    if os.path.exists(nameJSON):
        with open(nameJSON, 'r') as inputfile:
            inputDataDict =  json.load(inputfile)
        #更新
        for key_i in inputDataDict.keys():
            if key_i in dictJSON.keys():
                inputDataDict[key_i].update(dictJSON[key_i])
        inputDataDict.update(dictJSON)
        #儲存
        with open(nameJSON, 'w') as outfile:
            json.dump(inputDataDict, outfile, sort_keys=True)
    else:
        with open(nameJSON, 'w') as outfile:
            json.dump(dictJSON, outfile, sort_keys=True)
    return

In [25]:
updateAndSaveJSON(inputDataJSON, data_dict)

In [19]:
# 更新地震的 date Sequence
with open(earthquakeDataJSON, 'w') as outfile:
    json.dump(earthquakeDataDict, outfile)